In [ ]:
import pandas as pd
import skimage
from sample_methods import df_tools, image_tools
from label_blurring import blur_utils

### Sample images

In [ ]:
lt = 5
ut = 100
#Create dataframe of imagelinks and taxanomic labels
data_df = df_tools.observations_to_df("example_dwc/occurrence.txt", "example_dwc/multimedia.txt")
data_df.dropna()
species_freq = data_df['species'].value_counts(sort=True)
species = species_freq.index
#Sample a subset of the observations
sample_df = df_tools.sample(data_df, ut, lt, 'species', species, species_freq)

### Download Sample

In [ ]:
sample_species = sample_df['species'].unique()
image_tools.build_image_dir(sample_species, "example_image_folder")
image_tools.download_sample("example_image_folder", sample_df, 'species', data_df)

### Blurring Labels

In [ ]:
image_tools.build_image_dir(sample_species, "example_blurred_image_folder")
model = "label_blurring/CRAFT_models/craft_mlt_25k.pth" # Model weights need to be downloaded

for label in species:

  #Loop through training images
  try:
    path = "example_image_folder/train/{}".format(label)
    print("Path: {}".format(path))
    target_folder = "example_blurred_image_folder/train/{}/".format(label)
    blur_utils.CRAFT_Blur_directory(model, path, margin=15, res_dirname=target_folder, cuda=True)
    print("{} finished".format(target_folder))
  except Exception as e:
    print("Error: failed to open train folder for {} \n {}".format(label, e))

### Separate holdout test images

In [ ]:
image_tools.split_test_images("example_blurred_image_folder", sample_species)